In [1]:
%pip install pandas
import pandas as pd

%pip install numpy
import numpy as np

%pip install requests
import requests

import sqlite3
from datetime import datetime


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


### Alerta

In [2]:
%pip install plyer
from plyer import notification

def alerta(nivel, base, etapa):
    if etapa == "inicio": # Para o incio do processo da pipeline
        title = "Pipeline ETL Iniciado"
        message = "Iniciando"

    elif etapa == "fim": # Para o fim do processo da pipeline
        title = "Pipeline ETL Concluído"
        message = "Finalizado"
        
    elif nivel == 0:
        title = "Etapa Concluída"
        message = f"{etapa} da {base} concluída com sucesso."
        
    elif nivel == 1:
        title = "Alerta Baixo"
        message = f"Falha no carregamento {base} na etapa {etapa}"

    elif nivel == 2:
        title = "Alerta Moderado"
        message = f"Falha no carregamento {base} na etapa {etapa}"
    
    elif nivel == 3:
        title = "Alerta Alto"
        message = f"Falha no carregamento {base} na etapa {etapa}"

    else:
        title = "Alerta desconhecido" #Para situaçao de nivel de alerta fora do proposto.
        message = f"Nivel de alerta desconhecido ({nivel}) para {base} na etapa {etapa}"

    notification.notify(
        title=title,
        message=message,
        app_name = "Projeto Final",
        timeout=10
    )


#adicionar a funcao alerta(nivel, base, etapa) em cada etapa

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 1 Extração API

In [3]:
#def get_json_api(url): #funçao comentada para rodar automaticamente. Alterar para a funçao ser chamada em parte do codigo
    #'''
    #    Request GET url e retorna json de saida
    #'''
etapa = "Extraçao"
base = "API"

try:
    response = requests.get(url = "https://brasilapi.com.br/api/banks/v1")

    if response.status_code == 200:
        data_json = response.json()
        alerta(0,base,etapa) #Sucesso na etapa de extracao da API
        data_json
        #return data_json #caso inserido como funçao
    
    # Erro
    else:
        alerta(2,base,etapa) #Falha na etapa de extracao da API
        raise Exception(f"erro request,  {response.status_code} - {url}\n{response.text}")

except Exception as e:
    alerta(3,base,etapa) #erro critico na etapa
    raise e
    

### 1.1 Base Brasil Banks_url

In [4]:
df = pd.DataFrame()
url = "https://brasilapi.com.br/api/banks/v1"
response = requests.get (url)
response.status_code

200

In [4]:
if response.status_code == 200:
    data_json = response.json()
else: 
    print ("erro ao acessar a API")

In [6]:
#data_json

In [5]:
banks_full = pd.DataFrame(data_json)
#banks_full.head(60)

banks_full.columns

Index(['ispb', 'name', 'code', 'fullName'], dtype='object')

In [6]:
ispb = [ispb for ispb in banks_full['ispb']]
name =[name for name in banks_full['name']]
Code = [code for code in banks_full['code']]
fulname = [fn for fn in banks_full ['fullName']]
fulname = [fn for fn in banks_full ['fullName']]

### Tabela 1 - ISPB E NAME

In [8]:
base = "tabelas ISPB e Name"
etapa = "Extraçao"

try:
    banks_full= pd.DataFrame ({'ispb':ispb, 'name': name})
    banks_full = pd.DataFrame (banks_full)
    
    alerta(0,base,etapa) # Sucesso na etapa de extracao

    #banks_full
    #banks_full.head(60)
except Exception as e:
    alerta(2,base,etapa) #falha na extraçao

### TABELA 2 - CODE & FULLNAME

In [10]:
base = "tabelas Code e FullName"
etapa = "Extraçao"

try:
    banks_full= pd.DataFrame ({'code':Code, 'fulname':fulname})
    banks_full = pd.DataFrame (banks_full)
    alerta(0,base,etapa) # Sucesso na etapa de extracao

    #banks_full
    #banks_full.head(60)

except Exception as e:
    alerta(2,base,etapa) #falha na extraçao


### TABELA 3 - ISPB & CODE

In [11]:
base = "tabelas ISPB e Code"
etapa = "Extraçao"

try:
    banks_full= pd.DataFrame ({'ispb':ispb, 'code': Code})
    banks_full = pd.DataFrame (banks_full)
    alerta(0,base,etapa) # Sucesso na etapa de extracao

    #banks_full
    #banks_full.head(60)
    
except Exception as e:
    alerta(2,base,etapa) #falha na extraçao


In [12]:

conn = sqlite3.connect('Projeto.db')

df1 = pd.DataFrame({'ispb':ispb, 'name': name})
df2 = pd.DataFrame({'code':Code, 'fulname':fulname})
df3 = pd.DataFrame({'ispb':ispb, 'code': Code})

try:
    df1.to_sql(name='tabela1', con=conn, if_exists='replace', index=False)
    df2.to_sql(name='tabela2', con=conn, if_exists='replace', index=False)
    df3.to_sql(name='tabela3', con=conn, if_exists='replace', index=False)

    print("Tabelas foram criadas com sucesso no banco de dados.")

except Exception as e:
    print(f"Ocorreu um erro ao salvar as tabelas: {str(e)}")

finally:

    conn.close()


Tabelas foram criadas com sucesso no banco de dados.


In [14]:

conn = sqlite3.connect('Projeto.db')

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

print("Tabelas existentes no banco de dados:")
for table in tables:
    print(table[0])

cursor.close()
conn.close()


Tabelas existentes no banco de dados:
tabela1_backup
tabela2_backup
tabela3_backup
tabela1
tabela2
tabela3


In [16]:


try:
    conn = sqlite3.connect('Projeto.db')
    cursor = conn.cursor()

    tabelas_para_excluir = ['name', 'fulname', 'code', 'banks_full']

    for tabela in tabelas_para_excluir:
        cursor.execute(f"DROP TABLE IF EXISTS {tabela}")

    conn.commit()

    print("Tabelas excluídas com sucesso.")

except sqlite3.Error as e:
    print(f"Ocorreu um erro ao excluir as tabelas: {e}")

finally:

    if cursor is not None:
        cursor.close()

    if conn is not None:
        conn.close()


Tabelas excluídas com sucesso.


In [18]:

try:

    conn = sqlite3.connect('Projeto.db')

    df_tabela1 = pd.read_sql_query("SELECT * FROM tabela1 ORDER BY name", conn)

    # Exibir os dados como tabela
    print("\nDados da tabela1 ordenados por name:")
    display(df_tabela1)  # Utilize display() para mostrar o DataFrame como uma tabela formatada

except sqlite3.Error as e:
    print(f"Ocorreu um erro durante as operações: {e}")

finally:
    # Fechar a conexão se estiver aberta
    if conn is not None:
        conn.close()



Dados da tabela1 ordenados por name:


,ispb,name
0,,None
1,,None
2,37715993,ACCREDITO SCD S.A.
3,13140088,ACESSO SOLUÇÕES DE PAGAMENTO S.A. - INSTITUIÇÃ...
4,92856905,ADVANCED CC LTDA
...,...,...
354,05351887,ZEMA CFI S/A
355,37414009,ZIPDIN SCD S.A.
356,00329598,ÍNDIGO INVESTIMENTOS DTVM LTDA.
357,13293225,ÓRAMA DTVM S.A.


In [19]:

try:
    conn = sqlite3.connect('Projeto.db')

    df_tabela2 = pd.read_sql_query("SELECT * FROM tabela2 ORDER BY Fulname", conn)

    print("\nDados da tabela2 ordenados por Fulname:")
    display(df_tabela2) 

except sqlite3.Error as e:
    print(f"Ocorreu um erro durante as operações: {e}")

finally:
    if conn is not None:
        conn.close()



Dados da tabela2 ordenados por Fulname:


,code,fulname
0,NaN,None
1,NaN,None
2,406.0,ACCREDITO - SOCIEDADE DE CRÉDITO DIRETO S.A.
3,332.0,ACESSO SOLUÇÕES DE PAGAMENTO S.A. - INSTITUIÇÃ...
4,117.0,ADVANCED CORRETORA DE CÂMBIO LTDA
...,...,...
354,359.0,"ZEMA CRÉDITO, FINANCIAMENTO E INVESTIMENTO S/A"
355,418.0,ZIPDIN SOLUÇÕES DIGITAIS SOCIEDADE DE CRÉDITO ...
356,407.0,ÍNDIGO INVESTIMENTOS DISTRIBUIDORA DE TÍTULOS ...
357,355.0,ÓTIMO SOCIEDADE DE CRÉDITO DIRETO S.A.


In [20]:

try:
    conn = sqlite3.connect('Projeto.db')

    cursor = conn.cursor()

    cursor.execute("""
        DELETE FROM tabela1
        WHERE ROWID NOT IN (
            SELECT MIN(ROWID)
            FROM tabela1
            GROUP BY ispb, name
        )
    """)

    cursor.execute("""
        DELETE FROM tabela2
        WHERE ROWID NOT IN (
            SELECT MIN(ROWID)
            FROM tabela2
            GROUP BY code, fulname
        )
    """)

    cursor.execute("""
        DELETE FROM tabela3
        WHERE ROWID NOT IN (
            SELECT MIN(ROWID)
            FROM tabela3
            GROUP BY ispb, code
        )
    """)
    
    conn.commit()
    print("Dados duplicados removidos das três tabelas com sucesso.")

except sqlite3.Error as e:
    print(f"Ocorreu um erro ao remover dados duplicados: {e}")

finally:

    if cursor is not None:
        cursor.close()

    if conn is not None:
        conn.close()


Dados duplicados removidos das três tabelas com sucesso.


In [21]:
try:
    conn = sqlite3.connect('Projeto.db')

    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE tabela1_backup AS
        SELECT *
        FROM tabela1
    """)

    conn.commit()
    print("\nBackup da tabela1 criado.")

except sqlite3.Error as e:
    print(f"Ocorreu um erro ao criar o backup da tabela1: {e}")

finally:
    if cursor is not None:
        cursor.close()

    if conn is not None:
        conn.close()


Ocorreu um erro ao criar o backup da tabela1: table tabela1_backup already exists


In [22]:
try:
    conn = sqlite3.connect('Projeto.db')

    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE tabela2_backup AS
        SELECT *
        FROM tabela2
    """)

    conn.commit()
    print("\nBackup da tabela2 criado.")

except sqlite3.Error as e:
    print(f"Ocorreu um erro ao criar o backup da tabela2: {e}")

finally:
    if cursor is not None:
        cursor.close()

    if conn is not None:
        conn.close()

Ocorreu um erro ao criar o backup da tabela2: table tabela2_backup already exists


In [23]:
try:
    conn = sqlite3.connect('Projeto.db')

    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE tabela3_backup AS
        SELECT *
        FROM tabela3
    """)

    conn.commit()
    print("\nBackup da tabela3 criado.")

except sqlite3.Error as e:
    print(f"Ocorreu um erro ao criar o backup da tabela3: {e}")

finally:
    if cursor is not None:
        cursor.close()

    if conn is not None:
        conn.close()

Ocorreu um erro ao criar o backup da tabela3: table tabela3_backup already exists


In [24]:

try:
    conn = sqlite3.connect('Projeto.db')

    cursor = conn.cursor()

    cursor.execute("""
        SELECT t1.name, t1.ispb, t3.code
        FROM tabela1 t1
        INNER JOIN tabela3 t3 ON t1.ispb = t3.ispb
    """)
    joined_table1_t3 = cursor.fetchall()
    print("Junção entre tabela1 e tabela3:")
    for row in joined_table1_t3:
        print(row)

    cursor.execute("""
        SELECT t2.fulname, t2.code, t3.ispb
        FROM tabela2 t2
        INNER JOIN tabela3 t3 ON t2.code = t3.code
    """)
    joined_table2_t3 = cursor.fetchall()
    print("\nJunção entre tabela2 e tabela3:")
    for row in joined_table2_t3:
        print(row)

    conn.commit()
    print("\nOperações de junção concluídas.")

except sqlite3.Error as e:
    print(f"Ocorreu um erro durante as operações: {e}")

finally:

    if cursor is not None:
        cursor.close()

    if conn is not None:
        conn.close()


Junção entre tabela1 e tabela3:
('BCO DO BRASIL S.A.', '00000000', 1.0)
('BRB - BCO DE BRASILIA S.A.', '00000208', 70.0)
('Selic', '00038121', None)
('Bacen', '00038166', None)
('SANTINVEST S.A. - CFI', '00122327', 539.0)
('CCR SEARA', '00204963', 430.0)
('AGK CC S.A.', '00250699', 272.0)
('CONF NAC COOP CENTRAIS UNICRED', '00315557', 136.0)
('ÍNDIGO INVESTIMENTOS DTVM LTDA.', '00329598', 407.0)
('CAIXA ECONOMICA FEDERAL', '00360305', 104.0)
('STN', '00394460', None)
('BANCO INTER', '00416968', 77.0)
('COLUNA S.A. DTVM', '00460065', 423.0)
('BCO RIBEIRAO PRETO S.A.', '00517645', 741.0)
('BANCO BARI S.A.', '00556603', 330.0)
('BCO CETELEM S.A.', '00558456', 739.0)
('EWALLY IP S.A.', '00714671', 534.0)
('BANCO SEMEAR', '00795423', 743.0)
('PLANNER CV S.A.', '00806535', 100.0)
('FDO GARANTIDOR CRÉDITOS', '00954288', 541.0)
('BCO B3 S.A.', '00997185', 96.0)
('BCO RABOBANK INTL BRASIL S.A.', '01023570', 747.0)
('CIELO IP S.A.', '01027058', 362.0)
('CCR DE ABELARDO LUZ', '01073966', 322.0)
(